# DTM

## Setting

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import ldaseqmodel
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

stemmer = PorterStemmer()
keyword_num = 6  # 키워드 개수 (수정할 수 있는 부분)
data_size = 1200  # 데이터 크기, 각 키워드 별로 200개씩 키워드 6개이므로 년도별 1200개(수정할 수 있는 부분)
topic_num = 50  # Topic의 개수 (수정할 수 있는 부분)

## Sentence tokenize

In [ ]:
keyword_tokenized_list = []  # 각 년도별로 tokenized 결과 저장 (년도별로 총 n개씩 저장됨)
for year in range(2017, 2022):
  for i in range(0, keyword_num * 2, 2):
    data = pd.read_excel(str(year) + "_result.xlsx")
    data.drop(['Unnamed: 0'], axis=1, inplace=True)  # Index 표시 제거

    keyword = data.columns[i]
    keyword_data = data[keyword].tolist()

    for j in range(len(keyword_data)):
      temp = []
      tokenized_sentence = word_tokenize(keyword_data[j])
      for word in pos_tag(tokenized_sentence):
        if ('NN' in word[1]) and (word[0] not in stopwords.words('english')):
            # 단어 앞 뒤에 필요 없는 기호 제거
            text = word[0].strip('\\')
            text = text.strip('$')
            text = text.strip('-')
            text = text.strip('#')
            text = text.strip('^')
            text = text.strip('/')
            text = text.strip('%')
            text = text.strip('.')
            text = text.strip('_')
            text = text.strip('~')
            text = text.strip('=')
            text = text.strip('+')
            text = text.strip('*')
            text = text.strip(',')
            if(text == ""):
              continue
            temp.append(text)

      keyword_tokenized_list.append(temp)

## DTM

In [ ]:
my_dict = Dictionary(keyword_tokenized_list)
corpus = [my_dict.doc2bow(text) for text in keyword_tokenized_list]

# 2017 ~ 2021이므로 총 5개 (1200개씩)
time_slice = [data_size, data_size, data_size, data_size, data_size]
# num_topics : tuning해야하는 hyperparameter (토픽 수를 얼마나 설정할 것인지)
ldaseq = ldaseqmodel.LdaSeqModel(
    corpus=corpus, id2word=my_dict, time_slice=time_slice, num_topics=topic_num)

for i in range(topic_num):
  df = pd.DataFrame()
  for j in range(5):
    li = ldaseq.print_topic_times(i, top_terms=len(my_dict))[j]
    li.sort()  # Sorting data
    df[j + 2017] = [val[1] for val in li]

    if(j == 0):
      df.index = [val[0] for val in li]

  df.to_excel("topic" + str(i + 1) + "_dtm.xlsx")